%md
# Analisis de Sentimientos - Comentarios
#### Procesamiento del Lenguaje Natural - NLP - LLMS (IA)<br>
Utilización de Modelos del lenguaje grande para la clasificación de sentimientos y librerías especializadas para trabajar texto


In [0]:
#%restart_python

#Se instalan las siguiente librerias y paquetes 
%pip install nltk
%pip install spacy
%pip install --upgrade typing_extensions
%pip install plotnine
%pip install wordcloud
!python -m spacy download es_core_news_sm


In [0]:
import pandas as pd
import numpy as np

reviews = pd.read_csv('film_reviews_result.csv', sep= "|", )

In [0]:
reviews.sample(5)

Este dataset (corpus) contiene criticas realizadas por los usuarios de www.filmaffinity.com sobre todas las películas y series españolas (Mas de 10000 peliculas).   


El dataset (copus) esta formado por:    

film_name: Título de la película.     
gender: Genero de la película (comedia, terror, acción, etc.)         
film_avg_rate: Nota media de la película (votos de todos los usuarios)      
review_rate: Nota que el usuario que hace la crítica pone a la película.      
review_title: Título de la crítica.     
review_text: Crítica de la película     

In [0]:
#examinacion de valores nulo

reviews.info()

#### Funcion de prosesamiento de datos


Se construye una función con el cual el objetivo de la misma Es hacer un proceso de limpieza de datos Para obtener un análisis descriptivo De los comentarios de las películas

Como tal se instalan las bibliotecas necesarias para la construcción Dado que se trabaja en un proceso Interactivo el data fin es un objeto pandas El cual una vez procesada esa información nos entregará El comentario toquenizado y lematizado Con esto podemos saber cuáles son las palabras más frecuentes dentro de todo el Corpus de los del comentario y asimismo dentro de toda la data ¿Qué palabras más frecuentes se utiliza? Inclusive se puede observar por género de película y es lo que procederemos a revisar

In [0]:
import re, string
from nltk.corpus import stopwords
import spacy
import nltk
nltk.download('stopwords') #Descargamos stop works en el caso de que no este descargado

nlp_lematizar = spacy.load('es_core_news_sm') #reducir palabras a su forma base o lema (ej. “corriendo” → “correr”).

palabras_vacias = list(stopwords.words('spanish'))
print(palabras_vacias[1:10])

#Cosntruccion de funcion para trabajar cada comentario
coment = "Un ejemplo de !!!película con muy buenas críticas.,/*-😊😊8😊"

def limpieza(comentario, i = " "):
  texto = comentario.lower()
  texto = texto.encode('ascii', 'ignore').decode('ascii') #eliminamos caracteristicas raras emojis etc
  texto =  re.sub(f"[{re.escape(string.punctuation)}]", "", texto) #Eliminacion de signos de puntuacion
  texto =  re.sub(r"\d+", "", texto) #Eliminar nuemeros
  texto =  re.sub(r"\s+", " ", texto).strip() #normalizar espacios
  doc = nlp_lematizar(texto)  # tokeniza y lematiza  salida objeto Doc
  tokens = [i.lemma_ for i in doc if i.lemma_ not in palabras_vacias]
  return tokens

limpieza(coment)

# Se utilizará Este objeto con el cual Se eliminará aquellas palabras que no tienen un significado Representativo dentro del comentario

In [0]:
reviews2 = (
  reviews
  .assign(
    review_rate = lambda x: x['review_rate'].astype('float'),
    film_avg_rate = lambda x: x['film_avg_rate'].str.replace(",", ".").astype('float'),
    genero =  reviews['gender'].str.split(",").str[0], #De cada lista dentro de cada fila, dame el elemento en la posición 0
    tokenizacion =  lambda x :  x['review_text'].map(limpieza)
  )
)



In [0]:
reviews2 = (
  reviews2
  .assign(
    conteo_palabras = lambda x : x['tokenizacion'].str.len(),
  )
)

reviews2.sample(5)

In [0]:
#cuanta peliculas y comentarios se disponen

peliculas_comentarios = (
  reviews2
  .groupby('film_name', as_index=False)
  .size()
  .sort_values('size', ascending=False)
)
print("#Distribucion del numero de comentarios por pelicula",
      "\n"*2,
      peliculas_comentarios.describe(),
      "\n"*2,
      "TOP 14 - Films con mayor comentarios en el Data Set"
    )

peliculas_comentarios.reset_index(drop=True)[0:16]
#Aproximadamente exisyte 766 pelicual registradas
#vemo cuantos comentarios disponemos

Utilzaremos otro paque de python plotline ya que es la representacipon de ggplot de r en python ¡Es una excelente noticia! Para muchos analistas que vienen de R, descubrir plotnine es como volver a casa.

In [0]:
from plotnine import ggplot, aes, geom_histogram, labs, theme_light

# Creamos el gráfico
grafico_longitud = (
    ggplot(reviews2) 
    + aes(x='conteo_palabras') 
    + geom_histogram(bins=50, fill="#69b3a2", color="white")
    + labs(
        title="Distribución del número de palabras",
        subtitle="Análisis sobre 10,058 comentarios de data cruda",
        x="Cantidad de Palabras",
        y="Frecuencia"
    )
    + theme_light()
)
# Para mostrarlo
print("\n")
grafico_longitud

In [0]:
from plotnine import ggplot, aes, geom_col, labs, theme_light, coord_flip

#Gráfico de Calificación Promedio de la Película
p1 = (
    ggplot(reviews2) 
    + aes(x='film_avg_rate') 
    + geom_histogram(bins=15, fill="#392aa8", color="white")
    + labs(
        title="Puntuación General de Filmaffinity",
        subtitle="Promedio histórico calculado por la plataforma",
        x="Calificación (Escala 1-10)",
        y="Frecuencia de Películas"
    )
    + theme_light()
)

# Gráfico de Puntuación del Usuario
p2 = (
    ggplot(reviews2) 
    + aes(x='review_rate') 
    + geom_histogram(bins=10, fill="#7C7C93", color="white")
    + labs(
        title="Puntuación de las Reseñas",
        subtitle="Notas individuales otorgadas por los usuarios",
        x="Rating del Usuario (Escala 1-10)",
        y="Frecuencia de Votos"
    )
    + theme_light()
)

# 3. Unir los gráficos lado a lado sin usar los "ax" de Matplotlib (Evita el TypeError)
p1.show(), p2.show()

In [0]:
from plotnine import ggplot, aes, geom_col, labs, theme_light, coord_flip
generos = (
  reviews2
  .groupby("genero", as_index=False)
  .agg(
     {
         "film_name": "count"
     }
  )
  .sort_values("film_name", ascending=False)
  .assign(
      participacion_porce = lambda x : np.round((x["film_name"] / x["film_name"].sum())*100,2)
  )
)

p1 = (
    ggplot(generos)  # plotnine trabaja mejor con pandas
    + aes(x='reorder(genero, film_name)', y='film_name') #ordenamos el eje
    + geom_col(fill="#392aa8", color="white")
    + labs(
        title="Numero de videos por genero",
        x="Genero",
        y="Numero de Peliculas"
    )
    + theme_light()
    + coord_flip()  # opcional: mejora la lectura
)

p1.show()
generos

Dado que tenemos 10000 comentarios El proceso para realizar la obtención del sentimiento se lo hará a través de un modelo de lenguaje largo llms Utilizaremos los modelos que vienen por defecto en databriks Para hacer el procesamiento

Recordemos que nuestros Datos están almacenados en un objeto de pandas y si lo hacemos directamente utilizando El modelo de lenguaje grande Y el objeto Donde se almacena los comentarios pues tardará demasiado tiempo para tanto posito pues miraremos a Spark Para utilizar las funciones  De computación distribuida A través de Pyspark

### Analisis palabras -  Frecuencia por Genero

 Transformación objeto dataframe Pandas a un objeto dataframe de PySpark

In [0]:
import pyspark.sql.functions as f

df_spark = spark.createDataFrame(reviews2)
df_tokens = (
    df_spark
    .select(
        f.col("genero"),
        f.explode("tokenizacion").alias("palabra")
    )
)
df_tokens.count()
df_tokens.limit(10).display()


In [0]:
df_wc = (
    df_tokens
    .groupBy(f.col('genero'), f.col('palabra'))
    .agg(
       f.count( f.col('palabra')).alias("frecuencia")
    )
    .orderBy(f.desc("frecuencia"))
)
 
df_wc.count()
df_wc.limit(100).display()

In [0]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

df_wc_pd = df_wc.toPandas()
for genero in df_wc_pd["genero"].unique()[1:5]: #Selecionamos 4 generos para graficar
    subset = df_wc_pd[df_wc_pd["genero"] == genero]

    frecuencias = dict( #convierte palabras en un diccionario { "bueno": 12, "malo": 5,"excelente": 7}
        zip(subset["palabra"],
         subset["frecuencia"])
    )

    # nube de palabras grafica 
    wc = WordCloud(
        width=800,
        height=400,
        background_color="white",
        max_words=10,
        collocations=False
    ).generate_from_frequencies(frecuencias)

    plt.figure(figsize=(10,5))
    plt.imshow(wc, interpolation="bilinear")
    plt.axis("off")
    plt.title(f"WordCloud - {genero}")
    plt.show()

#### Obtención del sentimiento a través de un modelo de lenguaje grande

Dado que son más de 10000 comentarios que se dispone dentro de esta data será el ejercicio sólo para 150 comentarios utilizando un modelo de lenguaje grande esto con el afán de probar Las capacidades de extraccion del sentimiento en comentarios

Además para procesos aún más eficiente se utiliza pispar para la herramienta de procesamiento y utilizar así los clúster de distribución para que el trabajo no demore tanto tiempo